# Introduction

This lesson is the first of two that will show how to create maps in Python using Folium.

[Folium](https://python-visualization.github.io/folium/) is a wrapper that automates creating Leaflet maps. These maps (like Bokah graphs) are interactive: the user can zoom in and out, pan, etc., to explore the map.

The mapmaker doesn't need to work with HTML, CSS, or JavaScript: everything can be done within the Python ecosystem.

This makes it *much* easier to create a wide variety of different types of maps.

This lesson will show how to use Folium to create maps with points and circles, how to change their size and color, and how to annotate them with popup texts.

Recently, Folium and other tile providers changed their polices, so attribution for tile images must be provided. This can be done in the code (as done below) or via a helper function, [xyzservices](https://github.com/geopandas/xyzservices).

In [ ]:
"""
!pip3 install xyzservices
from xyzservices import TileProvider
#from xyzservices.lib import TileProvider
provider = TileProvider.from_qms("OpenTopoMap")
"""

In [ ]:
!pip3 install folium


In [ ]:
# import our tool libraries
import pandas as pd
import folium

According to this [Kaggle notebook](https://www.kaggle.com/code/alexisbcook/exercise-interactive-maps), adding

In [ ]:
"""
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')
"""

## Get the Data

Unlike many of our lessons, this one will draw on historical data, specifically a database of civil war battles.

The data we will be working with is [Jeffry Arnolds'](https://github.com/jrnold/acw_battle_data) [dataset](https://acw-battle-data.readthedocs.io/en/latest/). Arnold's data doesn't include lat/long data, so Karsdorp, Kestemont, and Allen added it as part of their "Narrating with Maps," chapter 7 of their excellent [*Humanities Data Analysis: Case Studies with Python*](https://www.amazon.com/Humanities-Data-Analysis-Studies-Python/dp/0691172366) (Princeton: Princeton University Press, 2021). Their code and data can be found at [Zenodo](https://zenodo.org/record/3563075).

I have taken the data developed as part of this chapter and saved it to my [Github Repo for this class](https://github.com/adamlporter/DataAnalysisClass). We will load the data from that repo.



In [ ]:
cw_df = pd.read_csv('https://raw.githubusercontent.com/adamlporter/DataAnalysisClass/master/cwsac_battle_locations_with_lat_long_tabdelim.csv',
                 sep = '\t',
                 parse_dates=['start_date','end_date'])

In [ ]:
cw_df.info()

The other dataset we will use is the Washington Post's Fatal Force database (it's the one we used earlier in the course when talking about statistics).

In [ ]:
ff_df = pd.read_csv('https://raw.githubusercontent.com/washingtonpost/data-police-shootings/master/v2/fatal-police-shootings-data.csv',parse_dates = ['date'])
ff_df = ff_df[ff_df['latitude'].notna()] # drop rows that do not have lat/lon data

## Draw a basic map

Let's start with a quick-and-dirty map.

Folium first requires us to initiate a Map object. We need to specify the center location of the map, the tile set we want to use, and a zoom level.

I generally center my maps on the center of the data. We can calculate this easily by finding the midpoint between the min and max values for lat / lon:
```
center = [ (df['lat'].max() + df['lat'].min())/2,
             (df['lon'].max() + df['lon'].min())/2]
```
### Select a basemap

Folium includes a number of [different tile sets](https://leaflet-extras.github.io/leaflet-providers/preview/) that will provide the basemap, upon which our data will be displayed. The default is `OpenStreetMap` but this doesn't make sense for a Civil War map. So we will use the `OpenTopoMap` map instead.

Finally, we need to specify a `zoom_start` value. This will vary depending on the size of the map.

We are going to be creating a lot of versions of the same map, so I'm going to create a function that initializes the map for us -- this will make our code easier to read and, if there is a problem, we only need to fix it once (in the function!).

Note that the default map is a topo (topological) map; if we a street map, we need to call it with `initMap('street')`.

In [ ]:
def initMap(type='topo'):
    if type = 'topo':
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}'
        attr = 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ, TomTom, Intermap, iPC, USGS, FAO, NPS, NRCAN, GeoBase, Kadaster NL, Ordnance Survey, Esri Japan, METI, Esri China (Hong Kong), and the GIS User Community'
    else:
        attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
        tiles = 'https://tile.openstreetmap.org/{z}/{x}/{y}.png'
    
    # find the geographic center of our map by looking for the min/max values of lat/lon and dividing by 2
    center = [ (df['lat'].max() + df['lat'].min())/2,(df['lon'].max() + df['lon'].min())/2]

    map = folium.Map(location=[40,-96], # roughly the center of the USA
               zoom_start = 5,
               attr=attr,
               tiles=tiles)
    return map

In the next cell, we initialize our map, , asssign it to the variable (`m`), and display it.

In [ ]:
m = initMap() # initialize the map using the function above

m # display the map

Folium creates is an HTML object, so the map is interactive. Look the map above: zoom in and move it around.

After we have initiated our map object, we can plot our data. Pandas gurus argue against using the `.iterrows()` because it is slow, but most maps use relatively small dataframes (there are less than 400 rows in the civil war battles DF), so speed isn't *that* important.

For our first map, we will just iterate over the dataframe and plot battle locations. Folium has a large number of different ways to plot data.

If we use the `.Marker` method and just provide location information (lat/lon coordinates), Folium will create a map with the familiar 'info-sign' icons.

In [ ]:
m = initMap()

# .iterrows() iterates over the DF, returning an index number and the row of data
# for each row, we can access the data by specifying the column(s) to be accessed

for idx,row in df.iterrows():
    folium.Marker(location = [row['lat'],row['lon']]
                         ).add_to(m)

m

#### Your Turn

In the next cell,
* initialize a street map (`m = initMap('street')`).
* plot 1000 points from the fatal force df: `for idx,row in ff_df.sample(1000).iterrows()`
    * Note that the ff_df variables are `latitude` and `longitude`, so you will need to modify the `location = ` part of the loop.



### Change Glyphs and Colors

Rather than having a blue pin with white circle, you can specify a different icon for the pin.
Folium supports bootstrap, so you specify an icon from the [glyph list](https://getbootstrap.com/docs/3.3/components/) to put on the marker.

You can also specify different colors from the [default list](https://python-visualization.github.io/folium/modules.html#Icon):
```
['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 
 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
 'white', 'pink', 'lightblue', 'lightgreen','gray', 'black', 'lightgray']
```

To see this, I have created lists of the glyphs (not all of them!) and colors; we will assign them randomly to the different pins:

In [ ]:
import random

m = initMap()

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
        'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
        'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
        'gray', 'black', 'lightgray']

icons = ['home','flag','time','road','cog','trash','lock','headphones',
         'tag','video','book','tags','tint','question_sign','info-sign',
         'screenshot','exclamation-sign','plane','leaf','fire','gift','ok-sign',
         'certificate','thumbs-up','heart-empty','paperclip']

for idx,row in df.iterrows():
    folium.Marker(location = [row.lat,row.lon],
                icon=folium.Icon(icon = random.choice(icons),
                                 color = random.choice(colors)),
                         ).add_to(m)
m

This map looks silly, which is inappropriate given the seriousness of the map. More Americans died in the Civil War than in any other war in which the US has fought.

Mapmakers should think carefully about the sorts of colors and glyphs that are appropriate for the topic.

#### Your Turn

In the next cell,
* initialize a street map (`m = initMap('street')`).
* plot 1000 points from the fatal force df, using `random.choice` options from the prior example.

### Popup and Tooltip: Additional Information

Folium allows us to add popup information to the markers by using either the `popup=` or `tooltip=` parameters.
* `popup=` requires the user to click on the point to see the information
* `tooltip=` will display the information as the user moves their pointer across the map

We can assign data from the dataframe directly, if it is a string. If the data we want to inclue in the popup is a number or date, we will need to do a little manipulation to turn it into a string. This is straight-forward with [f-string](https://realpython.com/python-f-strings/) formatting.

In [ ]:
m = initMap()

for idx,row in df.iterrows():
  popup_text = f"{row.campaign}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"

  folium.Marker(location = [row.lat,row.lon],
                icon=folium.Icon(icon='star',color = 'green'),
                popup = popup_text
                         ).add_to(m)

m

The F-string formatting gives mapmakers enormous control over the text that appears.

For example, the campaign name includes dates:
```
Sand Creek Campaign [November 1964]
```
But the database includes specific dates. We don't need to specify a temporary variable: the f-string allows us to modify the text in place:
```python
row.campaign.split('[')[0]
```
will split the campaign text string at the opening bracket ("["). This results in two text strings; we select the first one by specifying the first string with the ```[0]``` index.

If we want to boldface the labels for "Date" and "Casualities", we can use the HTML code (```<B>...</B>``` to boldface the text between the tags.

Doing both of these, we can create a new f-string:

```python
  popup_text = f"{row.campaign.split('[')[0]}\n<B>Date:</B> {row.start_date:%Y-%m-%d}\n<B>Casualities:</B> {row.casualties:,.0f}"
```

Finally, since Folium creates HTML code, to force a newline, we need to use `<br>` (=line break) instead of `\n`.
```python
  popup_text = f"{row.campaign.split('[')[0]}<br><B>Date:</B> {row.start_date:%Y-%m-%d}<br><B>Casualities:</B> {row.casualties:,.0f}"
```


I will redraw the map above with the new popup_text string. I will also change this to a `tool_tip`, so you can see the difference.

In [ ]:
m = initMap()

for idx,row in df.iterrows():
  popup_text = f"{row.campaign.split('[')[0]}<br><B>Date:</B> {row.start_date:%Y-%m-%d}<br><B>Casualities:</B> {row.casualties:,.0f}"

  folium.Marker(location = [row.lat,row.lon],
                icon=folium.Icon(icon='star',color = 'green'),
                tooltip = popup_text
                         ).add_to(m)

m

#### Your Turn

In the next cell,
* initialize a street map (`m = initMap('street')`).
* plot 1000 points from the fatal force df, with popup text.
    * Have the popup display at least two pieces of data you think are important. (Hint: you may need to add a cell to recall the different columns in the ff_df, with either `ff_df.columns` or `ff_df.info()`.


## Other Markers

There are situations where the markers used on the above maps will be perfectly adequate. One problem with them is that they are large and tend to overlie each other, making it hard to see some markers, epecially on crowded maps.

Folium has two other markers that work better in this situation: `.Circle()` and `.CircleMarker()`. The first is measured in meters; the second in pixels. If the use the `.Circle()` marker, when we zoom in/out ont he map, the marker will change; if the use `.CircleMarker()` the dot will remain the same size.

These markers have some of the same attributes (color and popup) as the `.Marker()` we used before, but additionally, we can specify
* `radius` - the size of the circle (in meters or pixels)
* `fill` - should the circle be filled in or not
* `fill_opacity` - how opaque should the circle be (this is useful if we have larger circles that might overlie each other)

In [ ]:
m = initMap()

for idx,row in df.iterrows():
    popup_text = f"{row.battle_name}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"
    folium.CircleMarker(location = [row.lat,row.lon],
                color = 'red',
                radius = 3,
                fill = True,
                fill_opacity = 1,
                tooltip = popup_text
                ).add_to(m)

m


#### Your Turn

In the next cell,
* initialize a street map (`m = initMap('street')`).
* plot 1000 points from the fatal force df, using the `.CircleMarker` function and the tool_tip information from your prior cell.

When designing a map, users should think about the various ways they can convey information to the viewer. The map above shows all the battles in the database. Users can click on the points to get information about the battle (date, casualities, etc.). But we can actually convey some of this information graphically.

For example, we can vary the size of the marker to indicate the number of casualities. We can also change the color of the marker to indicate the year of the battle.

The former is pretty straightforward: we can set the radius value based on the casuality figures. To ensure the circles are not too small, we can use the `max()` function to select from a default value and the casuality-calculated value.

The latter is a bit more complex: we need to define a `dictionary` with key/value pairs for the years and the colors we wish to use. When we iterate through the dataframe, the color will be set by looking up the values in the dictionary.

In [ ]:
m = initMap()

color_dict = {'1861':'red','1862':'blue','1863':'green','1864':'purple','1865':'orange'}

for idx,row in df.iterrows():
    popup_text = f"Battle Name: {row.battle_name}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"
    folium.CircleMarker(location = [row.lat,row.lon],
                radius = max(3,row.casualties/1000),
                color = color_dict[f"{row.start_date:%Y}"],
                fill = True,
                tooltip = popup_text
                ).add_to(m)

m

#### Your Turn

In the next cell,
* initialize a street map (`m = initMap('street')`).
* plot 1000 points from the fatal force df, with the `CircleMarker` but this time change the color based on the date of the killing.
    * You will need to change the `color_dict` to incldue the years from 2015 - 2024. I provided a list of all the default colors above.

## Filter by Year

This map conveys more information than the earlier one: we can see which battles had the most casualities and, by paying attention to the colors, get some idea of how the war progressed.

It might be easier for us to look at one year at a time. This is easy to do with Pandas: we just need to filter the dataframe to select the year we wish to map.

In [ ]:
import datetime

m = initMap()

year = 1863 # <= specify a year here and use it to filter the date field in the next two rows
filter = (df['start_date'] >= datetime.datetime.strptime(f"{year}-01-01",'%Y-%m-%d')) & \
            (df['start_date'] <= datetime.datetime.strptime(f"{year}-12-31",'%Y-%m-%d'))

for idx,row in df[filter].iterrows(): # <= the filtered DF will only have battles in the year specified
    popup_text = f"Battle Name: {row.battle_name}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"
    folium.CircleMarker(location = [row.lat,row.lon],
                radius = max(3,row.casualties/1000),
                color = 'red',
                fill = True,
                tooltip = popup_text
                ).add_to(m)

m

We could produce a series of maps, year by year, to show the different battles in the civil war.

Folium makes it possible for us to plot these all on the same map by specifying different layers.

In the following map, we loop through the years 1861 to 1855. For each year:
1. We define a layer as a `FeatureGroup()` with a specific name (the year value) and add it to the map.
1. We create a filtered version of the DF that includes only rows of data for the desired year.
1. We iterate through the filtered DF to create our markers. These are added **to the layer** (not to the map, as in earlier examples).
1. We have added a tool to add `LayerControl()` to the map. This is a little box in the upper right corner: if you click on it, it will display radio buttons for the different layers, so you can click on the one you want to display.

When we initialized the map, we told Folium **not** to have a basemap (`tiles = None`). This is because if we initialize the map with basetiles, it will appear in the Control box. But we don't want to be able to turn off the basemap! Instead, we add the basemap with the `TileLayer()` method, where we specify `control = False`, to prevent THIS layer from appearing in the control box.

In [ ]:
m = folium.Map(location=[40,-95],
               zoom_start = 5,
               tiles = None)

folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ, TomTom, Intermap, iPC, USGS, FAO, NPS, NRCAN, GeoBase, Kadaster NL, Ordnance Survey, Esri Japan, METI, Esri China (Hong Kong), and the GIS User Community',
                 overlay=True,
                 control=False).add_to(m)

for year in range(1861,1866):
    layer = folium.FeatureGroup(name = year,overlay = False).add_to(m)

    filter = (df['start_date'] >= datetime.datetime.strptime(f"{year}-01-01",'%Y-%m-%d')) & \
            (df['start_date'] <= datetime.datetime.strptime(f"{year}-12-31",'%Y-%m-%d'))

    for idx,row in df[filter].iterrows():
        popup_text = f"Battle Name: {row.battle_name}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"
        folium.CircleMarker(location = [row.lat,row.lon],
                    radius = row.casualties/1000,
                    color = 'red',
                    fill = True,
                    tooltip = popup_text
                    ).add_to(layer)

folium.LayerControl().add_to(m)

m


#### Your Turn

I've copied the above cell into the cell below. Edit it 
* to produce a layered map of **3000** points from the `ff_df`.
* change the `tiles` and `attr` to the following values:
    * attr = '\&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    * tiles = 'https://tile.openstreetmap.org/{z}/{x}/{y}.png'
* change the date range to select dates from 2015 to 2024.
* edit the popup text to use the values you used previously
* edit it to refer to the fatal force dataframe throughout. 

In [ ]:
m = folium.Map(location=[40,-95],
               zoom_start = 5,
               tiles = None)

folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ, TomTom, Intermap, iPC, USGS, FAO, NPS, NRCAN, GeoBase, Kadaster NL, Ordnance Survey, Esri Japan, METI, Esri China (Hong Kong), and the GIS User Community',
                 overlay=True,
                 control=False).add_to(m)

for year in range(1861,1866):
    layer = folium.FeatureGroup(name = year,overlay = False).add_to(m)

    filter = (df['start_date'] >= datetime.datetime.strptime(f"{year}-01-01",'%Y-%m-%d')) & \
            (df['start_date'] <= datetime.datetime.strptime(f"{year}-12-31",'%Y-%m-%d'))

    for idx,row in df[filter].iterrows():
        popup_text = f"Battle Name: {row.battle_name}<br>Date: {row.start_date:%Y-%m-%d}<br>Casualities: {row.casualties:,.0f}"
        folium.CircleMarker(location = [row.lat,row.lon],
                    radius = row.casualties/1000,
                    color = 'red',
                    fill = True,
                    tooltip = popup_text
                    ).add_to(layer)

folium.LayerControl().add_to(m)

m

## Group and Heatmap

Folium provides several other visualizations for mapmakers, notably a *cluster visualization* and a *heatmap visualization*. Because the Civil War data isn't particulary good for showing how these tools can be used (they excell when faced with many individual points), the rest of this notebook will work with the *Fatal Force* database.

### Group / Cluster Map

Folium will group items together and, as the user zooms in, ungroup them to show the individual points.

To use this, we need to import a special Folium tool and tell the system to add a `marker_cluster` layer to the map. (The variable name can, of course, be anything the mapmaker desires.)
```python
from folium.plugins import MarkerCluster
marker_cluster = folium.plugins.MarkerCluster().add_to(m)
```
When we iterate over the data, rather than adding the points to the map (as we did above), we add them to the `marker_cluster` layer.

We can add any of the markers we described above to the layer (`.Marker()`, `.Circle()`, `.CircleMarker()` etc.); as the user zooms in, these are the points that will be revealed.

Binder has been able to disply our Civil War data (400 rows) well, but it seems to have trouble with larger numbers of points. So we will sample the `ff_df` and display $3000$ points. (You can experiment with different values -- what's the biggest sample you can get without having Binder choke?)

In [ ]:
m = initMap('street')

from folium.plugins import MarkerCluster
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

for idx,row in ff_df.sample(3000).iterrows():
    lat, lon = row.latitude, row.longitude
    folium.CircleMarker(location=[lat,lon],
        color = 'red',
        radius = 3,
        fill = True,
        fill_opacity = 1
        ).add_to(marker_cluster)

m

We can also format the markers using the tools described above.

In [ ]:
m = initMap('street')

from folium.plugins import MarkerCluster
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

color_dict = {'2015':'red','2016':'orange','2017':'darkred',
              '2018': 'green','2019':'blue','2020':'purple',
              '2021':'white','2022':'black','2023':'darkgreen', '2024':'pink'}

for idx,row in ff_df.sample(3000).iterrows():
    popup_text = f"Year:{row.date:%Y}<br>Race:{row.race}<br>Age:{row.age:.0f}"
    lat, lon = row.latitude, row.longitude
    folium.Marker(location=[lat,lon],
                  icon=folium.Icon(color = color_dict[f"{row.date:%Y}"]
                                  ),
                  tooltip = popup_text,
                  ).add_to(marker_cluster)

m

#### Your Turn

The next two cell will create a new DF for you to experiment with: murders in Chicago. The data comes from the [Chicago Data Portal](https://data.cityofchicago.org/Public-Safety/Homicides/iyvd-p5ga/about_data). It has data from 2001 - 2024.

It also creates a new map initizilation function that zooms in on Chicago. 





In [ ]:
homicides = homicides = pd.read_csv('https://raw.githubusercontent.com/adamlporter/DataAnalysisClass/master/Homicides_20240412.csv')
homicides['dt'] = pd.to_datetime(homicides['Date'],format="%m/%d/%Y %I:%M:%S %p" ) # add column as datetime format


In [ ]:
def initChiMap():
    attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    tiles = 'https://tile.openstreetmap.org/{z}/{x}/{y}.png'
    
    # find the geographic center of our map by looking for the min/max values of lat/lon and dividing by 2
    center = [ (df['lat'].max() + df['lat'].min())/2,(df['lon'].max() + df['lon'].min())/2]

    map = folium.Map(location=[42.8,-87.7], # roughly the center of Chicago
               zoom_start = 6,
               attr=attr,
               tiles=tiles)
    return map

In the next cell, create a cluster map that displays **3000** records from the **homicides** dataframe. Note that this data base capitalizes `Latitude` and `Longitude`.

Be sure to initilize the map with the `initChiMap()` function. (If you don't the map will still work, but you'll need to zoom in on Chicago to see the data displayed.)

### Heatmap



The [heatmap](https://python-visualization.github.io/folium/plugins.html#folium-plugins) tool doesn't allow us to add points with information (as we did above). Instead, it takes just a list of lat/lon points and processes them to create the heatmap. So rather than iterating through the DF, we will just pass the list of lat/lon points to the tool.

In [ ]:
m = initMap2()

from folium.plugins import HeatMap

points = ff_df[['latitude','longitude']].sample(3000).values.tolist()

HeatMap(points,radius = 12).add_to(m)

m

#### Your Turn

In the next cell, create a heatmap of a sample of 3000 homicides in Chicago.


Folium can present a heatmap of data that changes over time. This doesn't work especially well for the fatal force data because the data is sporadic. It would work better for things like traffic flows along major highways, since there is almost always a low-level base usage, which peaks during rush-hour.

But we can see how to arrange the data so that -- should you get a data set that would work better -- you can use this tool.

The `HeatMapWithTime()` method expects to have two groups of data, of the same length.
* `data=` needs to be a list of points in the form [lat,lon]
* `index=` needs to be a list of dates

One way to create this is to set up a dictionary, using some [tools](https://docs.python.org/3/library/collections.html) from the collections library.

The following code creates a dictionary whose default value is a `list`. It iterates over the DF, but since it uses [`itertuples()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html), it returns the values as tuples

For each date (in this case, I've specified Year-Month), it creates a string version of the datetime ("2020-01") as the key and appends a list of lat/lon data for the month.

The data looks like this:
```python
            [('2015-01',
              [[47.246826, -123.121592],
               [45.4874214, -122.8916961],
               [37.694766, -97.280554],
               [39.380084, -76.820805]]),
             ('2015-02',
              [[40.273404, -76.712841],
               [34.417432, -117.176872],
               [35.917642, -77.54755],
               [33.619301, -114.450926]]),
             ('2015-03',
              [[34.043131, -118.244634],
               [29.704199, -95.621853],
               ...
```
It then uses the `OrderedDict()` tool to sort the dictionary on its first element (the date field).

In the HeatMapWithTime()
* `list(data.values())` pulls out the values from the dictionary -- the list of lat/lon values -- and turns them into a list.

* `list(data.keys())` does the same thing for the dictionary's keys, which is the dates.

(I found this solution on [Stackoverflow](https://stackoverflow.com/questions/64325958/heatmapwithtime-plugin-in-folium).)

In [ ]:
m = initMap2()

# https://stackoverflow.com/questions/64325958/heatmapwithtime-plugin-in-folium

from folium.plugins import HeatMapWithTime

from collections import defaultdict, OrderedDict

data = defaultdict(list)

for row in ff_df.itertuples():
  data[row.date.strftime("%Y-%m")].append([row.latitude,row.longitude])

data = OrderedDict(sorted(data.items(),key = lambda t: t[0]))

hm = HeatMapWithTime(data = list(data.values()),
                     index = list(data.keys()),
                     radius = 10,
                     auto_play = True)

hm.add_to(m)

m

#### Your Turn

Using the Chicago Homicide data, do the following:
1. Select 10 years of data (use a boolean mask to limit it) and create a map with multiple layers (one per year) using the `FeatureGroup` function.
1. Select a different 10 year period and create a map that displays the homicides with different color circles for each year.
